Cannot implement because fun Huber_score_unscaled not found

In [111]:
import scipy.io
import numpy as np

BS = scipy.io.loadmat('BS.mat',struct_as_record=False)['BS']
theta_init = scipy.io.loadmat('theta_init_mas')['theta_init'].flatten()
rekf = scipy.io.loadmat('rekf.mat',struct_as_record=False)['rekf']
parameter = scipy.io.loadmat('params_mas')['parameter']

In [112]:
rekf['P0']

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [94]:
rekf[0][0][0]
len(rekf[0][0])
rekf.tolist()

[[(array([[22500,     0,     0,     0,     0],
          [    0, 22500,     0,     0,     0],
          [    0,     0, 22500,     0,     0],
          [    0,     0,     0, 22500,     0],
          [    0,     0,     0,     0, 22500]], dtype=uint16), array([[1, 0],
          [0, 1]], dtype=uint8), array([[0.02, 0.  ],
          [0.  , 0.02],
          [0.2 , 0.  ],
          [0.  , 0.2 ]]), array([[1. , 0. , 0.2, 0. ],
          [0. , 1. , 0. , 0.2],
          [0. , 0. , 1. , 0. ],
          [0. , 0. , 0. , 1. ]]), array([[30,  0,  4, 10]], dtype=uint8), array([[2500,    0,    0,    0],
          [   0, 2500,    0,    0],
          [   0,    0,   36,    0],
          [   0,    0,    0,   36]], dtype=uint16), array([[2]], dtype=uint8), array([[0.0001]]), array([[25]], dtype=uint8), array(['asymmetric'], dtype='<U10'), array([[1.5]]), array([[3]], dtype=uint8), array([[0]], dtype=uint8), array([[1.73863863]]), array([[0, 0, 0, ..., 0, 0, 0]], dtype=uint8))]]

In [108]:
import numpy as np
import robustsp as rsp

def ekf_toa_Masreliez(r_ges, theta_init, BS, parameter=None):
    if parameter == None:
        # use default parameters
        print("parameters are set to default")
        sigma_v = 1
        M  = len(BS) # M numer of BS, N number of samples
        P0 = np.diag([100, 100, 10, 10]) # initial state covariance
        R  = 150**2 * np.diag(np.ones(M)) # measurement covariance
        Ts = 0.2 # sampling frequency
        A  = np.array([[1, 0, Ts, 0], \
                       [0, 1, 0, Ts], \
                       [0, 0, 1,  0], \
                       [0, 0, 0,  1]])
        Q  = sigma_v **2 *np.eye(2)
        G  = np.vstack([Ts**2/2*np.eye(2), Ts*np.eye(2) ])
    else:
        P0 = parameter['P0']
        R  = parameter['R']
        Q  = parameter['Q']
        G  = parameter['G']
        A  = parameter['A']
    x = BS[:,0]
    y = BS[:,1]
    
    M = len(x)
    N = len(r_ges[0,:])
    
    P = np.zeros((N,4,4))
    P[0,:,:] = P0
    th_hat = np.zeros((4,N))
    th_hat[:,0] = theta_init.flatten()
    th_hat_min = np.zeros([4,N])
    P_min = np.zeros([N,4,4])
    H = np.zeros((M,4))
    h_min = np.zeros(M)
    
    for kk in range(1,N):
        th_hat_min[:,kk] = A @ th_hat[:,kk-1]
        
        P_min[kk,:,:] = A@P[kk-1,:,:]@A.T + G@Q@G.T
        
        for ii in range(M):
            H[ii,:] = [(th_hat_min[0,kk]-x[ii])/\
           np.sqrt((th_hat_min[0,kk]-x[ii])**2 + (th_hat_min[1,kk]-y[ii])**2) ,\
           (th_hat_min[1,kk]-y[ii])/np.sqrt((th_hat_min[0,kk]-x[ii])**2 \
                   + (th_hat_min[1,kk]-y[ii])**2)\
           ,0,0]
            h_min[ii] = \
            np.sqrt((th_hat_min[0,kk]-x[ii])**2 \
                + (th_hat_min[1,kk]-y[ii])**2)

        S = H@P_min[kk,:,:]@H.T + R
            
        try:
            C = np.linalg.cholesky(S)
        except:
            print('matrix modified in Masreliez filter')
            S += 500000*np.eye(M)
            C = np.linalg.cholesky(S)
        C = C.T   

        nu = np.linalg.inv(C)@(r_ges[:,kk]-h_min)
        
        K = P_min[kk,:,:] @ H.T @ np.linalg.inv(C.T)
        
        if parameter['singlescore']:
            v, vp = rsp.asymmetric_tanh(nu, parameter['c1'], parameter['c2'], parameter['x1'])
        
        th_hat[:,kk] = th_hat_min[:, kk] + K@v
        
        P[kk,:,:] = (np.eye(4) - K@np.linalg.inv(C)@H*np.mean(vp)) @ P_min[kk,:,:]
            
    return th_hat, P_min, P, parameter

In [109]:
a, b, c, d = ekf_toa_Masreliez(parameter['measureddistances'][0,0], theta_init, BS, rekf)


c:\users\computer\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: elementwise == comparison failed and returning scalar instead; this will raise an error or perform elementwise comparison in the future.
  """


ValueError: setting an array element with a sequence.

In [95]:
b

array([[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00]],

       [[ 2.50144040e+03,  0.00000000e+00,  7.20400000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  2.50144040e+03,  0.00000000e+00,
          7.20400000e+00],
        [ 7.20400000e+00,  0.00000000e+00,  3.60400000e+01,
          0.00000000e+00],
        [ 0.00000000e+00,  7.20400000e+00,  0.00000000e+00,
          3.60400000e+01]],

       [[ 1.96569153e+03, -2.50472236e+01,  1.28615187e+01,
         -7.20929938e-02],
        [-2.50472236e+01,  1.95589811e+03, -7.20929938e-02,
          1.28333304e+01],
        [ 1.28615187e+01, -7.20929938e-02,  3.60755258e+01,
         -2.07504027e-04],
        [-7.20929938e-02,  1.28333304e+